In [176]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.quantization import prepare_qat, convert, QuantStub, DeQuantStub
from torch.quantization.observer import MinMaxObserver
from torch.quantization.fake_quantize import FakeQuantize
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np
import struct
import os

In [177]:
def quantize_tensor(tensor, num_bits=4, scale=None, zero_point=None):
    qmin = -(2 ** (num_bits - 1))      # For 4 bits: -8
    qmax = (2 ** (num_bits - 1)) - 1   # For 4 bits: 7

    if scale is None or zero_point is None:
        min_val, max_val = tensor.min(), tensor.max()
        if max_val == min_val:
            scale = 1.0
            zero_point = 0
        else:
            scale = (max_val - min_val) / float(qmax - qmin)
            zero_point = qmin - min_val / scale
            zero_point = int(torch.round(zero_point).item())
        zero_point = max(qmin, min(qmax, zero_point))
    quantized = tensor / scale + zero_point
    quantized = torch.round(quantized)
    quantized = torch.clamp(quantized, qmin, qmax)
    quantized = quantized.to(torch.int8)  # Use int8 for signed 8-bit integers
    return quantized, scale, zero_point



In [178]:
import struct
import os
def export_quantized_weights(model, output_dir='weights'):
    os.makedirs(output_dir, exist_ok=True)

    layer_idx = 0
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            layer_idx += 1

            # Get weights and biases
            weights = module.weight.data.cpu()
            biases = module.bias.data.cpu() if module.bias is not None else None

            # Flatten weights and biases and combine them
            if biases is not None:
                combined_tensor = torch.cat([weights.flatten(), biases.flatten()])
            else:
                combined_tensor = weights.flatten()

            # Compute min and max over the combined tensor
            min_val, max_val = combined_tensor.min(), combined_tensor.max()

            # Compute scale and zero_point
            qmin = -8  # For 4 bits
            qmax = 7
            if max_val == min_val:
                scale = 1.0
                zero_point = 0
            else:
                scale = (max_val - min_val) / float(qmax - qmin)
                zero_point = qmin - min_val / scale
                zero_point = int(torch.round(zero_point).item())
            zero_point = max(qmin, min(qmax, zero_point))

            # Quantize weights using the computed scale and zero_point
            quantized_weights, _, _ = quantize_tensor(weights, num_bits=4, scale=scale, zero_point=zero_point)

            # Convert quantized weights to uint8 for packing
            quantized_weights_uint = (quantized_weights & 0x0F).to(torch.uint8)

            # Flatten and pack weights
            flat_weights = quantized_weights_uint.flatten()
            if flat_weights.numel() % 2 != 0:
                flat_weights = torch.cat([flat_weights, torch.tensor([0], dtype=torch.uint8)])
            packed_weights = (flat_weights[0::2] << 4) | flat_weights[1::2]
            packed_weights = packed_weights.numpy()

            # Save packed weights to a COE file
            weight_file = os.path.join(output_dir, f'fc{layer_idx}_weight.coe')
            with open(weight_file, 'w') as f:
                f.write('memory_initialization_radix=16;\n')
                f.write('memory_initialization_vector=\n')
                hex_values = [f'{byte:02X}' for byte in packed_weights]
                f.write(',\n'.join(hex_values))
                f.write(';')
            print(f"Saved quantized weights for layer {layer_idx} to {weight_file}")

            # Quantize biases using the same scale and zero_point
            if biases is not None:
                quantized_biases, _, _ = quantize_tensor(biases, num_bits=4, scale=scale, zero_point=zero_point)
                quantized_biases_uint = (quantized_biases & 0x0F).to(torch.uint8)
                quantized_biases_uint = quantized_biases_uint.numpy()

                # Save biases to a COE file
                bias_file = os.path.join(output_dir, f'fc{layer_idx}_bias.coe')
                with open(bias_file, 'w') as f:
                    f.write('memory_initialization_radix=16;\n')
                    f.write('memory_initialization_vector=\n')
                    hex_values = [f'{bias_value:02X}' for bias_value in quantized_biases_uint]
                    f.write(',\n'.join(hex_values))
                    f.write(';')
                print(f"Saved quantized biases for layer {layer_idx} to {bias_file}")
            else:
                print(f"No bias in layer {layer_idx}")

            # Save the common scale and zero point
            params_file = os.path.join(output_dir, f'layer_{layer_idx}_params.txt')
            with open(params_file, 'w') as f:
                f.write(f'scale: {scale}\n')
                f.write(f'zero_point: {zero_point}\n')
            print(f"Saved scale and zero point for layer {layer_idx} to {params_file}")



In [179]:
# Custom 4-bit Observer
class MinMaxObserver4bit(MinMaxObserver):
    def __init__(self, dtype=torch.qint8, qscheme=torch.per_tensor_affine, reduce_range=False, quant_min=-8, quant_max=7, **kwargs):
        super(MinMaxObserver4bit, self).__init__(dtype=dtype, qscheme=qscheme, reduce_range=reduce_range, quant_min=quant_min, quant_max=quant_max, **kwargs)



In [180]:
# Custom 4-bit Fake Quantization Module
class FakeQuantize4bit(FakeQuantize):
    def __init__(self, observer=MinMaxObserver4bit, quant_min=-8, quant_max=7, dtype=torch.qint8, qscheme=torch.per_tensor_affine, reduce_range=False, **kwargs):
        super(FakeQuantize4bit, self).__init__(
            observer=observer,
            quant_min=quant_min,
            quant_max=quant_max,
            dtype=dtype,
            qscheme=qscheme,
            reduce_range=reduce_range,
            **kwargs
        )



In [181]:
class MinMaxObserver32bit(MinMaxObserver):
    def __init__(self, dtype=torch.qint32, qscheme=torch.per_tensor_affine, reduce_range=False, **kwargs):
        super(MinMaxObserver32bit, self).__init__(dtype=dtype, qscheme=qscheme, reduce_range=reduce_range, **kwargs)
        self.quant_min = -2**31
        self.quant_max = 2**31 - 1

In [182]:
class FakeQuantize32bit(FakeQuantize):
    def __init__(self, observer=MinMaxObserver32bit, quant_min=-2**31, quant_max=2**31 - 1, dtype=torch.qint32, qscheme=torch.per_tensor_affine, reduce_range=False, **kwargs):
        super(FakeQuantize32bit, self).__init__(
            observer=observer,
            quant_min=quant_min,
            quant_max=quant_max,
            dtype=dtype,
            qscheme=qscheme,
            reduce_range=reduce_range,
            **kwargs
        )

In [183]:
def apply_4bit_qat(model):
    # Define custom QConfig with 4-bit fake quantization
    custom_qconfig = torch.quantization.QConfig(
        activation=FakeQuantize32bit.with_args(observer=MinMaxObserver32bit),
        weight=FakeQuantize4bit.with_args(observer=MinMaxObserver4bit, quant_min=-8, quant_max=7)
    )
    model.qconfig = custom_qconfig

    # Prepare the model for QAT
    prepare_qat(model, inplace=True)
    return model


In [184]:
class quantComplexMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, hidden_size5, hidden_size6, hidden_size7, hidden_size8, output_size):
        super(quantComplexMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.bn1 = nn.BatchNorm1d(hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.bn2 = nn.BatchNorm1d(hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.bn3 = nn.BatchNorm1d(hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.bn4 = nn.BatchNorm1d(hidden_size4)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(hidden_size4, hidden_size5)
        self.bn5 = nn.BatchNorm1d(hidden_size5)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(hidden_size5, hidden_size6)
        self.bn6 = nn.BatchNorm1d(hidden_size6)
        self.relu6 = nn.ReLU()
        self.fc7 = nn.Linear(hidden_size6, hidden_size7)
        self.bn7 = nn.BatchNorm1d(hidden_size7)
        self.relu7 = nn.ReLU()
        self.fc8 = nn.Linear(hidden_size7, hidden_size8)
        self.bn8 = nn.BatchNorm1d(hidden_size8)
        self.relu8 = nn.ReLU()
        self.fc9 = nn.Linear(hidden_size8, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        x = self.bn6(x)
        x = self.relu6(x)
        x = self.fc7(x)
        x = self.bn7(x)
        x = self.relu7(x)
        x = self.fc8(x)
        x = self.bn8(x)
        x = self.relu8(x)
        x = self.fc9(x)
        return x


In [185]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [199]:
    # Device Configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load Data
    df = pd.read_csv('dataset1.csv')


    # For demonstration, let's create random data
    input_size = 27   # Adjust based on your actual data
    output_size = 6   # Adjust based on your actual data
    num_samples = 1000

    train_data_columns = df.columns[:27]
    train_label_columns = df.columns[27:]  # 只选择第一个output的列
    print(df[train_data_columns].dtypes)
    train_data = torch.tensor(df[train_data_columns].values, dtype=torch.float32).to(device)
    train_labels = torch.tensor(df[train_label_columns].values, dtype=torch.float32).to(device)
    for i in range(100-1):
        train_labels[i][0] = (train_labels[i][0] * 99)
        train_labels[i][1] = (train_labels[i][1] * 99)
        train_labels[i][2] = (train_labels[i][2] * 99)
        train_labels[i][3] = (train_labels[i][3] * 99)
        train_labels[i][4] = (train_labels[i][4] * 99)
        train_labels[i][5] = (train_labels[i][5] * 99)

    

    train_dataset = TensorDataset(train_data, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)

    # Define Model
    #hidden_sizes = [512, 1024, 1024, 1024, 1024, 512, 512, 256]
    model = quantComplexMLP(input_size, 512, 1024, 1024, 1024, 1024, 512, 512, 256, output_size).to(device)

   # Apply Custom 4-bit QAT
    #model = apply_4bit_qat(model)

    # Define Loss and Optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.03)

    # Train the Model
    num_epochs = 3000  # Adjust as needed
    train_model(model, train_loader, criterion, optimizer, num_epochs)

    export_quantized_weights(model)



Input_0     int64
Input_1     int64
Input_2     int64
Input_3     int64
Input_4     int64
Input_5     int64
Input_6     int64
Input_7     int64
Input_8     int64
Input_9     int64
Input_10    int64
Input_11    int64
Input_12    int64
Input_13    int64
Input_14    int64
Input_15    int64
Input_16    int64
Input_17    int64
Input_18    int64
Input_19    int64
Input_20    int64
Input_21    int64
Input_22    int64
Input_23    int64
Input_24    int64
Input_25    int64
Input_26    int64
dtype: object
Epoch [1/3000], Loss: 3202675456.0000
Epoch [2/3000], Loss: 3202556416.0000
Epoch [3/3000], Loss: 3202393856.0000
Epoch [4/3000], Loss: 3202072320.0000
Epoch [5/3000], Loss: 3201672704.0000
Epoch [6/3000], Loss: 3201259776.0000
Epoch [7/3000], Loss: 3200823552.0000
Epoch [8/3000], Loss: 3200361472.0000
Epoch [9/3000], Loss: 3199853824.0000
Epoch [10/3000], Loss: 3199313408.0000
Epoch [11/3000], Loss: 3198724608.0000
Epoch [12/3000], Loss: 3198117376.0000
Epoch [13/3000], Loss: 3197429504.0000
Ep

In [193]:
model

quantComplexMLP(
  (fc1): Linear(in_features=27, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=512, out_features=1024, bias=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=1024, out_features=1024, bias=True)
  (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=1024, out_features=1024, bias=True)
  (bn4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (fc5): Linear(in_features=1024, out_features=1024, bias=True)
  (bn5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu5): ReLU()
  (fc6): Linear(in_features=1024, out_features=512, bias=True)
  (bn6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=Tr

In [200]:
test_data_columns = df.columns[:input_size]
# Preprocess the input data for inference
test_data = df[test_data_columns].values[:5] 

# Convert to tensor and send to device
test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)

model.eval()  

with torch.no_grad():
    outputs = model(test_data_tensor)

# Convert the outputs back to CPU and numpy for interpretation
predicted_outputs = outputs.cpu().numpy()
print(f"output: {test_out}")
# Display the input and output
for i, (input_sample, output_sample) in enumerate(zip(test_data, predicted_outputs)):
    print(f"Input {i+1}: {input_sample}")
    
    print(f"Output {i+1}: {output_sample}")

    84172, -62805

output: Target_1
Input 1: [ -3838   3208   4815  -7305   6274  10225 -26275  41177  27282   2089
   3387   4355   6191   3507   7220   4545 -11293   9543   2703   1994
   3383   3820  -3052   4308   2953  -4864   4688]
Output 1: [ 92590.89    8245.949  42610.21  -27907.754 -34191.17  -87828.26 ]
Input 2: [ 8084 -8388  7723  2375  4568 -8298 -4767  2683 -5010  6827  7562  9324
  5928 -6200 -6543 -3901 -4269 -4617  4548  5250  4074  4703 -3112 -2981
  2071 -3698 -3190]
Output 2: [ 12602.124  21450.086   6940.834 -88741.43   30117.943  25678.18 ]
Input 3: [ -5811  -8151  -6229   7844  -5430  -5788   3645   5232  -2511 -13851
   8516 -12756 -15505  21240  18457  -7431   6370   4038   8048   5936
  -3913  -5424 -11431   7630  -6838  -5644   3774]
Output 3: [ 33065.9    54532.703  61766.664 -72320.87  -63897.83   19109.496]
Input 4: [  3319  -6346  -5967 -10336 -13020   3951   7558  -8733   6379  -6063
  -3525   2177  -7593   8790   8619   7475   6700   6663  -4378   3529
   2023  -3884   55

In [201]:
def process_coe_file(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Extracting the data section from the .coe file (assuming it's a comma-separated list)
    data = []
    in_data_section = False
    for line in lines:
        if "memory_initialization_vector=" in line:
            in_data_section = True
            # Extract the part after 'memory_initialization_vector='
            data.extend(line.split('=')[1].strip().split(','))
        elif in_data_section:
            if ';' in line:
                data.extend(line.replace(';', '').strip().split(','))
                break
            else:
                data.extend(line.strip().split(','))

    # Clean up empty strings from the list
    data = [d.strip() for d in data if d.strip()]

    # Insert '0' and 'end' after every 27 data entries
    processed_data = []
    for i in range(0, len(data), 27):
        processed_data.extend(data[i:i+27])
        processed_data.append('0')  # Insert 0

    # Recreate the .coe file format with the processed data
    with open(output_file, 'w') as f:
        # Write the header back (if needed)
        for line in lines:
            if "memory_initialization_vector=" in line:
                f.write("memory_initialization_vector=\n")
                break
            f.write(line)

        # Write the processed data
        for i, d in enumerate(processed_data):
            f.write(d)
            if i < len(processed_data) - 1:
                f.write(',')
            if (i + 1) % 10 == 0:  # Optional formatting: 10 entries per line
                f.write('\n')

        f.write(';')  # End the coe file

    print(f"Processed .coe file saved as {output_file}")

In [202]:
input_file = 'weights/fc1_weight.coe'
output_file = 'weights/fc1_weight_padding.coe'
process_coe_file(input_file, output_file)

Processed .coe file saved as weights/fc1_weight_padding.coe
